# particle scattering forward model



In [1]:
import pandas as pd
import math
import scipy.constants as const
import numpy as np

In [2]:
txt_dir = '../../Retrieval/unchanged/part_scat_forward_model'

## readfile='B6pr_2_dimensional_and_mass_oriented_24.0GHz.txt'
model = 'B6pr_2_dimensional_and_mass_oriented_24.0GHz.txt'

## readfile='readfile='SP_dimensional_and_mass_oriented_24.0GHz.txt'
#model = 'SP_dimensional_and_mass_oriented_24.0GHz.txt'

## readfile='HC_dimensional_and_mass_oriented_24.0GHz.txt'
#model = 'HC_dimensional_and_mass_oriented_24.0GHz.txt'

In [3]:
# in vsnow_hmrr
nlayers = 14        # every 200m 
nbin = nlayers

#IWCpsd = 

#def sub24hnum(slp2, y_sim, IWCpsd, nbin, deltaz)

In [4]:
# read in scattering model

# readfile='B6pr_2_dimensional_and_mass_oriented_24.0GHz.txt'
scat_mod = pd.read_csv('%s/%s' %(txt_dir,model),\
                    header=3,  names = ['r_ev','mass', 'C_bk', 'C_sca', 'C_ext', 'Ldim','none'], \
                    sep = '\s+', engine= 'python')


In [5]:
nx = nbin + nbin

# Rayleigh approximation?:
radar_freq = 24.0e9                  # radar frequency in [Hz]
Pi = const.pi                        # Pi
c = const.speed_of_light             # speed of light in [m/s]
K_sq_water = 0.92                    # dielectric factor abs(K)^2_liquid = 0.92       (AOS740 7-9)
wavel_mm = (c/radar_freq) / 1000.    # wavelength, lambda in [mm] 

In [20]:
#### READ IN BASE PARTICLE PROPERTIES ###########
#Ldim = scat_mod['Ldim']          # [um]
r_ev = scat_mod['r_ev']/1000.    # effective radius [um] --> [mm]
Ldim = r_ev/1000.                # [mm]
C_bk = scat_mod['C_bk']          # backscatter coefficient [m^2]
#C_ext = scat_mod['C_ext']        # extinction coefficient [m^2]
C_scat = scat_mod['C_sca']       # scattering coefficient [m^2]
mass = scat_mod['mass']*1000.    # mass [kg]  -->
#area = scat_mod['Ldim']          # [um]
#maxd = 

In [21]:
## in retrieval:
C_ext = C_scat

In [22]:
#### MAKE SIMPLE LOG PSD ###########

## in retrieval:
#Numd = PSD
#IWCpsd = IWC

In [23]:
# in vsnow_hmrr

# line 72: 
# for i=0,1439 do begin
#     readf, 3, s1, s2            ; reads in time and temp from Met Norway data
#     stemp(i)=s2
MetNo_dir = '../../Retrieval/new_20180102_copy/snowret'
MetNo = pd.read_csv('%s/Haukeli_surftemp_20161225.dat' %(MetNo_dir),\
                header=None, names = ['time','sfc_temp'], sep = '\s+', engine= 'python')
stemp = MetNo['sfc_temp']


# num_kazr = 1440
# line 109:
# for icol2 = 0, num_kazr-1 do begin          ; start retrieval loop for number of daily MRR profiles (1440)
# line 269:
# tguess=stemp(icol2)+273.0

#num_kazr = stemp.shape[0]
tguess = stemp+273.0


# line 272:
# for ilapse=0,nlayers-1 do begin
#     t_apriori(ilapse)=tguess-1.0*float(ilapse)              ;  ~ moist adiabat 5K/km 1K/200m

t_apriori = []
for ilapse in range(0, nlayers):
    t_apriori.append(tguess - 1.*ilapse)


# line 283:
# for iss=0, nlayers-1 do begin
#     ap_slp(iss)= -(0.03053)*(t_apriori(iss)-273.0)-0.08258        ; a priori log(lambda,ap)
#     ap_n0(iss)=  -(0.07193)* (t_apriori(iss)-273)+2.665           ; a priori log(N0,ap)

ap_slp = []
ap_N0 = []
for iss in range(0, nlayers):
    ap_slp.append( - (0.03053)* (t_apriori[iss] - 273.) - 0.08258)
    ap_N0.append( - (0.07193) * (t_apriori[iss] - 273.) + 2.665)



# line 301:
# for nkbins=0, nlayers-1 do begin
#       slp(nkbins)=ap_slp(nkbins)
# for nkbins=nlayers, nx-1 do begin
#       slp(nkbins)=ap_n0(nkbins-nlayers)

slp = []
for nkbins in range(0, nlayers):
    slp.append(ap_slp[nkbins])
for nkbins in range(nlayers, nx):
    slp.append(ap_N0[ nkbins - nlayers])



perturbation = 0.2           #; % value to perturb the retrieved rainrate
                             #; in calculation of the K-matrix
# line 346: upperturb=slp
# line 350: 
# for i= 0 , nx-1 do begin
#      upperturb(i) = slp(i)*(1.0 + perturbation/100.0)
upperturb = []
for i in range(0,nx):
    upperturb.append(slp[i] * ( 1. + perturbation/100.))
    


# line 360: sub24hnum, upperturb, y_max, iwcpsd, nlayers, delta_z

# line 582:
# endfor; icol2

In [24]:
# line 360: sub24hnum, upperturb, y_max, iwcpsd, nlayers, delta_z

slp2 = upperturb

snow_ext_accum = 0.
# line 106 in sub24hnum.pro:
# for i=0,nbin-1 do begin
#     lamb(i)=10^slp2(i)
#     n0(i)=10^slp2(i+nbin)    ;  3.0

lamb = []
N0 = []
for i in range(0, nbin):
    lamb.append(10.**(slp2[i]))
    N0.append(10**(slp2[i+nbin]))
    


In [25]:
numd = []
for j in range(0, nbin):
    for i in range(0, r_ev.shape[0]):
        numd.append(N0[j]*np.exp( - lamb[j] * Ldim[i]))
        
    ### integrate to find IWP/SWC in each layer- radar bin based upon PSD #########
    # line 128: 
#    iwcpsd(j)=numd(0)*mass(0)*(ldim(0)-0.)/2.

In [37]:
IWC_psd = []
for j in range(0, nbin):
    IWC_psd.append(numd[0] * mass[0] * (Ldim[0] - 0.)/2.)
    

In [39]:
pd.DataFrame(IWC_psd[0])

,sfc_temp
0,2.385636e-09
1,2.389599e-09
2,2.389599e-09
3,2.397543e-09
4,2.401525e-09
5,2.405514e-09
6,2.397543e-09
7,2.397543e-09
8,2.401525e-09
9,2.393568e-09


In [38]:
pd.DataFrame(IWC_psd)

,0,1,2,3,4,5,6,7,8,9,...,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09
sfc_temp,2.385636e-09,2.389599e-09,2.389599e-09,2.397543e-09,2.401525e-09,2.405514e-09,2.397543e-09,2.397543e-09,2.401525e-09,2.393568e-09,...,1.784337e-09,1.772531e-09,1.772531e-09,1.778424e-09,1.766658e-09,1.787300e-09,1.799204e-09,1.802193e-09,1.799204e-09,1.811187e-09


In [ ]:
# line 130:
#for i=0,nd-2 do begin
#    iwcpsd(j)=iwcpsd(j)+(numd(i)*mass(i)+numd(i+1)*mass(i+1))*(ldim(i+1)-ldim(i))/2.0

